# Ejercicio Práctico de Regresión

**Objetivo y descripción del problema**

El **objetivo** del problema es implementar modelos de regresión para 3 datasets distintos, evaluando la bondad de sus predicciones y cuantificando los errores producidos por los modelos.

Para ello, se van a seguir los siguientes pasos:

- Comprensión del problema y carga de datos
- Preprocesado y análisis inicial de los datos
- Partición externa de los datos
- Selección de atributos
- Estandarización de atriutos
- Validación cruzada para optimizar los hiperparámetros
- Modelado con modelos de regresión (OLS y KNN Regressor)
- Evaluación de los resultados

**Datasets**

Se cargan los datasets con los que se va a trabajar:

**Advertising**
- Conjunto de datos sobre el gasto de diversos anuncios en campañas publicitarias en diferentes medios.
- Predicción: ventas conseguidas según el gasto invertido en publicidad
- Features:
 - TV: gasto en TV
 - Radio: gasto en Radio
 - Newspaper: gasto en Newspaper

**Airfoil**

- Conjunto de datos de la NASA, obtenido de una serie de pruebas aerodinámicas y acústicas de secciones de palas aerodinámicas bidimensionales y tridimensionales realizadas en un túnel de viento anecoico. El dataset comprende superficies aerodinámicas NACA 0012 de diferentes tamaños a varias velocidades de túnel de viento y ángulos de ataque. El tramo del perfil aerodinámico y la posición del observador fueron los mismos en todos los experimentos. 
- Predicción: nivel de presión sonora en decibelios
- Features:
 - Frequency, in Hertzs.
 - Angle of attack, in degrees.
 - Chord length, in meters.
 - Free-stream velocity, in meters per second.
 - Suction side displacement thickness, in meters. 


**California housing**
- Conjunto de datos de las diferentes características que presenta el vino así como la calidad del mismo.
- Predicción: value (valor de la casa)
- Features:
 - MedInc
 - HouseAge
 - AveRooms	
 - AveBedrms	
 - Population	
 - AveOccup	
 - Latitude	
 - Longitude	



In [ ]:
import numpy as np
import pandas as pd
import os

from sklearn import datasets, linear_model
import warnings
warnings.filterwarnings("ignore")

from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import mutual_info_regression,f_regression
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import cross_validate, KFold, cross_val_predict, train_test_split, cross_val_score

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor

from sklearn import preprocessing

In [ ]:

ruta = os.path.join("datasets", "Advertising.csv")
fichero = open(ruta)
data_1 = pd.read_csv(ruta)
fichero.close()

ruta = os.path.join("datasets", "airfoil_self_noise.dat")
fichero = open(ruta)
data_2 = pd.read_csv(ruta, sep="\t", names = ["frequency", "angle_of_attack", "chord_length", 
                                                 "free_stream_velocity", "Suction_thickness", "pressure_level"])
fichero.close()

# Carga de datos.
dataset = datasets.fetch_california_housing(as_frame=True)
data_3 = dataset.data
data_3['Value'] = dataset.target

print(np.shape(data_3))

In [ ]:
datasets_dic = {'data_1': data_1,'data_2': data_2, 'data_3':data_3}

Se hacen varios chequeos y se aplican técnicas de preprocesado de datos para mejorar la comprensión y estructura de los datasets.

**Tratamiento de valores faltantes**

In [ ]:
# Conteo de valores faltantes

print("---- Advertising ----")
print(data_1.isnull().sum())
print('---- Airfoil ----')
print(data_2.isnull().sum())
print('---- California ----')
print(data_3.isnull().sum())

In [ ]:
# Advertising
data_1["Radio"].fillna(data_1["Radio"].mean(),inplace=True)
data_1["Newspaper"].fillna(data_1["Newspaper"].mean(),inplace=True)


**Outliers**

Definir una función para estudiar los outliers.

**Separación de atributos y variable target**

Para que los cálculos sean más sencillos, vamos a crear dos diccionarios, uno con los atributos de cada uno de los datasets y otro con las variables target.

In [ ]:
X = {
    'ADS': data_1.iloc[:,:-1],
    'AIR': data_2.iloc[:,:-1],
    'CALIFORNIA': data_3.iloc[:,:-1]
}

y = {
    'ADS': data_1.iloc[:,-1],
    'AIR': data_2.iloc[:,-1],
    'CALIFORNIA': data_3.iloc[:,-1]
}

In [ ]:
# Se comprueba que se ha hecho bien con el tamaño de los datasets
for nombre, exp in X.items():
    print(nombre)
    print(exp.shape)

**Partición hold out**

In [ ]:
X_train= {}
y_train = {}
X_test = {}
y_test = {}

for nombre, exp in X.items():
    X_train[nombre],X_test[nombre],y_train[nombre],y_test[nombre] = train_test_split(X[nombre],y[nombre], 
                                                                                     test_size=0.2, random_state=42)

In [ ]:
print(" ------------------------")
for key,item in X_train.items():
    print(key)
    print(item.shape)
print(" ------------------------")
for key,item in y_train.items():
    print(key)
    print(item.shape)
print(" ------------------------")
for key,item in X_test.items():
    print(key)
    print(item.shape)
print(" ------------------------")
for key,item in X_test.items():
    print(key)
    print(item.shape)

**Selección de atributos**

In [ ]:
# Comprobación del tamaño de los datasets de entrenamiento
for key,item in X_train.items():
    print(key)
    print(item.shape)

In [ ]:
Variance_T = {}
X_train_sel = {}
for nombre, exp in X_train.items():
    selector = VarianceThreshold(0.3)
    Variance_T[nombre] = selector.fit(X_train[nombre])
    X_train_sel[nombre] = Variance_T[nombre].transform(X_train[nombre])

In [ ]:
for key,item in X_train_sel.items():
    print(key)
    print(item.shape)

**Estandarización de los datos**

In [ ]:
for key,item in X_train.items():
    print(key)
    print(item.shape)

In [ ]:
standardizer = {}
X_train_std = {}

for nombre, exp in X_train_sel.items():
    estandarizador = preprocessing.StandardScaler()
    standardizer[nombre] = estandarizador.fit(X_train_sel[nombre])
    X_train_std[nombre] = standardizer[nombre].transform(X_train_sel[nombre])

In [ ]:
standardizer

In [ ]:
for key,item in X_train_std.items():
    print(key)
    print(item.shape)

**Comparación de modelos con cross_val_score**

In [ ]:
# Se definen los algoritmos dentro de un diccionario y se hacen distintas ejecuciones cambiando los hiperparámetros
# del modelo hasta conseguir aquellos que den mejor resultado

k=5
algoritmos = {'OLS' : LinearRegression(),

              'KNN' : KNeighborsRegressor(n_neighbors = k, weights='distance', metric='euclidean', algorithm='kd_tree'),
             }

Se hace validación cruzada con 10 bolsas y se obtiene la metrica MAE para ambos algoritmos

In [ ]:
resultados = {}

for nombre, exp in X_train_std.items():
    resultados[str('OLS_')+nombre] = cross_val_score(algoritmos['OLS'], X_train_std[nombre], y_train[nombre], 
                                                     cv = KFold(n_splits=10, shuffle=True, random_state=42),
                                                     scoring='neg_mean_absolute_error')
    print("OLS", nombre, "cross_val_MAE:   %0.4f +/- %0.4f" % (-resultados[str('OLS_')+nombre].mean(), 
                                                                resultados[str('OLS_')+nombre].std()))

    resultados[str('KNN')+nombre] = cross_val_score(algoritmos['KNN'], X_train_std[nombre], y_train[nombre], 
                                                     cv = KFold(n_splits=10, shuffle=True, random_state=42),
                                                     scoring='neg_mean_absolute_error')
    print("KNN", nombre, "cross_val_MAE:   %0.4f +/- %0.4f" % ( -resultados[str('KNN')+nombre].mean(), 
                                                                 resultados[str('KNN')+nombre].std()))

**Modelo final**

En el paso anterior se ha utilizado validación cruzada para optimizar los hiperpámetros, y ahora se va a entrenar el modelo con todos los datos de entrenamiento. Los resultados se guardan en un diccionario.

In [ ]:
for nombre, exp in X_train_std.items():
    print(exp.shape)

In [ ]:
modelo_definitivo_ols = {}
modelo_definitivo_knn = {}

for nombre, exp in X_train_std.items():
    modelo_definitivo_ols[nombre + str('_OLS')] = algoritmos['OLS'].fit(X_train_std[nombre], y_train[nombre])
    
for nombre, exp in X_train_std.items():
    modelo_definitivo_knn[nombre + str('_KNN')] = algoritmos['KNN'].fit(X_train_std[nombre], y_train[nombre])

In [ ]:
y_train_pred_ols = {}
y_train_pred_knn = {}
for nombre, exp in X_train_std.items():
    y_train_pred_ols[nombre + str('_OLS')] = cross_val_predict(modelo_definitivo_ols[nombre + str('_OLS')], 
                                         X_train_std[nombre], y_train[nombre], 
                                         cv=KFold(n_splits=10, shuffle=True, random_state=42))

for nombre, exp in X_train_std.items():
    y_train_pred_knn[nombre + str('_KNN')] = cross_val_predict(modelo_definitivo_knn[nombre + str('_KNN')], 
                                         X_train_std[nombre], y_train[nombre], 
                                         cv=KFold(n_splits=10, shuffle=True, random_state=42))

In [ ]:
# Se comprueba el tamaño
for nombre, exp in y_train_pred_ols.items():
    print(nombre)
    print(exp.shape)

In [ ]:
# Se comprueba el tamaño
for nombre, exp in y_train_pred_knn.items():
    print(nombre)
    print(exp.shape)

**Estandarización y selección de atributos del conjunto de test**

In [ ]:
for nombre, exp in X_test.items():
    print(nombre)
    print(exp.shape)

In [ ]:
for nombre, exp in X_train.items():
    print(nombre)
    print(exp.shape)

In [ ]:
X_test_sel = {}
for nombre, exp in X_test.items():
    X_test_sel[nombre] = Variance_T[nombre].transform(X_test[nombre])

In [ ]:
X_test_std = {}
for nombre, exp in X_test_sel.items():
    X_test_std[nombre] = standardizer[nombre].transform(X_test_sel[nombre])

In [ ]:
for nombre, exp in X_test_std.items():
    print(nombre)
    print(exp.shape)

In [ ]:
# Se utilizan los modelos de KNN y OLS entrenados con el conjunto de entrenamiento para hacer predicciones con el conjunto 
# de test y evaluar los modelos creados.

y_test_pred_ols = {}
y_test_pred_knn = {}
for nombre, exp in X_test_std.items():
    y_test_pred_ols[nombre + str('_OLS')] = cross_val_predict(modelo_definitivo_ols[nombre + str('_OLS')], 
                                         X_test_std[nombre], y_test[nombre], 
                                         cv=KFold(n_splits=5, shuffle=True, random_state=42))


for nombre, exp in X_test_std.items():
    y_test_pred_knn[nombre + str('_KNN')] = cross_val_predict(modelo_definitivo_knn[nombre + str('_KNN')], 
                                         X_test_std[nombre], y_test[nombre], 
                                         cv=KFold(n_splits=5, shuffle=True, random_state=42))

**Métricas: MAE, RMSE, MAPE**

In [ ]:
import sklearn.metrics as metrics

def evaluacion(y_true, y_pred, metricas):
    res = {}
    for nombre, funcion in metricas.items():
        res[nombre] = funcion(y_true, y_pred)
    return res


metricas = {
  'MAE':  metrics.mean_absolute_error,
  'RMSE': lambda y, y_pred:
          math.sqrt(metrics.mean_squared_error(y, y_pred)),
  'MAPE': lambda y, y_pred:
          np.mean(np.abs((y - y_pred) / y)) * 100,
  'R2':   metrics.r2_score}

**Evaluación del conjunto de entrenamiento**

In [ ]:
# Métricas correspondientes a LinearRegression()

import math
for nombre, exp in y_train.items():
    print(nombre + str('_OLS'))
    MAE =   metricas['MAE'](y_train[nombre], y_train_pred_ols[nombre + str('_OLS')])
    RMSE = metricas['RMSE'](y_train[nombre], y_train_pred_ols[nombre + str('_OLS')])
    MAPE = metricas['MAPE'](y_train[nombre], y_train_pred_ols[nombre + str('_OLS')])
    R2 = metricas['R2'](y_train[nombre], y_train_pred_ols[nombre + str('_OLS')])
    
    print('MAE:  %.4f' % MAE)
    print('RMSE: %.4f' % RMSE)
    print('MAPE: %.4f' % MAPE)
    print('R2: %.4f' % R2)

    print('------------------')

In [ ]:
# Evaluación del algoritmo KNN y presentación de resultados.
for nombre, alg in y_train.items():
    results = evaluacion(y_train[nombre], y_train_pred_knn[nombre + str('_KNN')], metricas)


**Evaluación del conjunto de test**

In [ ]:
for nombre, exp in y_test.items():
    print(nombre + str('_OLS'))
    MAE =   metricas['MAE'](y_test[nombre], y_test_pred_ols[nombre + str('_OLS')])
    RMSE = metricas['RMSE'](y_test[nombre], y_test_pred_ols[nombre + str('_OLS')])
    MAPE = metricas['MAPE'](y_test[nombre],y_test_pred_ols[nombre + str('_OLS')])
    R2 =     metricas['R2'](y_test[nombre], y_test_pred_ols[nombre + str('_OLS')])

    print('MAE:  %.4f' % MAE)
    print('RMSE: %.4f' % RMSE)
    print('MAPE: %.4f' % MAPE)
    print('R2:   %.4f' % R2)
    print('-------------------------------')

In [ ]:
# Evaluación y presentación de resultados.
for nombre, alg in y_test.items():
    results = evaluacion(y_test[nombre], y_test_pred_knn[nombre + str('_KNN')], metricas)
   

**Conclusion**

Explicar los resultados obtenidos.